In [1]:
import pandas as pd 
import mlflow


import numpy as np
import sklearn.metrics as metrics



In [3]:
def hackathon_loss(y_true, y_pred, remittence):
    cond_linear = np.logical_and(y_pred>y_true, remittence==0)
    cond_quadratic = ~np.logical_and(y_pred>y_true, remittence==0)
    diff = y_pred-y_true
    return np.hstack([diff[cond_linear], diff[cond_quadratic]**2]).mean()



In [4]:
def get_metrics(y_true, y_pred, remittence):
    mean_absolute_error = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    r2 = metrics.r2_score(y_true, y_pred)
    hachathon_loss = hackathon_loss(y_true, y_pred, remittence)

    return {
        "MAE": mean_absolute_error,
        "MSE": mse,
        "r2": r2,
        "hackathon_loss": hachathon_loss,
    }

def make_lags(df, lags, col):
    
    for lag in lags:
         df = pd.merge(
                 left=df, 
                 right=df.groupby(["CISLO_ODBERATELE", "ID_ADRESA"])[col].shift(lag), 
                 left_index=True, 
                 right_index=True, 
                 suffixes=("", f"_NEW_SHIFT_{lag}"), 
                 how="left"
             )
            
    return df


In [ ]:
pip install catboost

In [5]:
from catboost import CatBoostRegressor

In [6]:
test_data = pd.read_csv("in/tables/REFL_TEST.csv")

In [7]:
reflex_test = test_data.copy()

In [8]:
monitoring_reflex = pd.read_csv("in/tables/MONITORING_SUM_REFL-csv.csv")

In [9]:
reflex_test.EXPEDICE.min()

'2021-07-01'

In [10]:
#REMITENDA ends at2022-06-16
reflex_test.groupby(["EXPEDICE"]).agg({"REMITENDA":"sum"}).head(15)

,REMITENDA
EXPEDICE,
2021-07-01,13574
2021-07-08,14029
2021-07-15,15396
2021-07-22,13006
2021-07-29,14182
2021-08-05,14222
2021-08-12,12544
2021-08-19,13266
2021-08-26,15063


In [11]:
#take the first month for testing
reflex_test = reflex_test[(reflex_test["EXPEDICE"]>="2021-07-01") & (reflex_test["EXPEDICE"]<="2021-07-29")]

In [ ]:
columns_filter_for_CATBoost = ["RETEZEC", "CROSSPROMO","GECO","WEEK","HDS","SOCIAL","TV","OTHER",
                               "SALES_LAG_4","SALES_LAG_5","SALES_LAG_6","SALES_LAG_7","SALES_LAG_8","SALES_LAG_9",
                               "SALES_LAG_10","SALES_LAG_11","SALES_LAG_4_AVG_TILL_NOW","REMITENDA","SALES","MONITORING_NEW_SHIFT_2"]


In [13]:
reflex_test = reflex_test.dropna(subset=["SALES_LAG_4","SALES_LAG_5","SALES_LAG_6","SALES_LAG_7","SALES_LAG_8","SALES_LAG_9",
                               "SALES_LAG_10","SALES_LAG_11"])

## Try only POS with monitoring

In [14]:
unique_pos_ids = monitoring_reflex.CISLO_ODBERATELE.unique()

In [15]:
reflex_test = reflex_test[reflex_test["CISLO_ODBERATELE"].isin(unique_pos_ids)]

In [16]:
reflex_test = reflex_test.rename(columns={"DATE_y":"DATE"})

In [17]:
reflex_test = pd.merge(reflex_test,monitoring_reflex,how="left",
                      on=["DATE","TITUL","CISLO_ODBERATELE"])

In [18]:
reflex_test = make_lags(reflex_test, lags=[2],col="MONITORING")

In [19]:
reflex_test = reflex_test.dropna(subset=["MONITORING_NEW_SHIFT_2"])

In [20]:
reflex_test = reflex_test.filter(columns_filter_for_CATBoost)

In [21]:
remittence_test = reflex_test.REMITENDA

In [22]:
y_true = reflex_test.SALES

In [23]:
df_MA_model_copy = reflex_test.copy()
df_MA_model_copy["ROLLING_MEAN"] = df_MA_model_copy[["SALES_LAG_4","SALES_LAG_5","SALES_LAG_6","SALES_LAG_7"]].mean(axis=1)
MA_predictions = df_MA_model_copy.ROLLING_MEAN.round().values

In [24]:
reflex_test = reflex_test.drop(columns=["REMITENDA","SALES"])

In [25]:
logged_model = 'runs:/60daaf753b1d4eb6817ce07965dd149f/pipeline'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
y_pred = loaded_model.predict(pd.DataFrame(reflex_test))

y_pred = y_pred.round()

In [26]:
test_metrics_model = get_metrics(
        y_true=y_true, 
        y_pred=y_pred, 
        remittence=remittence_test)
    
test_metrics_model

{'MAE': 1.656078860898138,
 'MSE': 11.048740416210295,
 'r2': 0.8311256476135767,
 'hackathon_loss': 11.041621029572838}

In [28]:
test_metrics_MA = get_metrics(
        y_true=y_true, 
        y_pred=MA_predictions, 
        remittence=remittence_test)
    
test_metrics_MA

{'MAE': 1.8127053669222344,
 'MSE': 15.429901423877327,
 'r2': 0.7641618399758375,
 'hackathon_loss': 15.429901423877327}

In [163]:
with mlflow.start_run(run_name = "TEST_SET_PERFORMANCE_MONITORING", experiment_id=1) as run:
    
    test_metrics_model = get_metrics(
        y_true=y_true, 
        y_pred=y_pred, 
        remittence=remittence_test)
    
    test_metrics_MA = get_metrics(
        y_true=y_true, 
        y_pred=MA_predictions, 
        remittence=remittence_test)
    
    features_dict = dict(zip(list(reflex_test.columns),list(["feature" for i in range(len(reflex_test.columns))])))
    
    mlflow.log_params(features_dict)
    mlflow.log_metrics({f"test_model_{metric_name}_rounded":metric_value for metric_name, metric_value in test_metrics_model.items()})
    mlflow.log_metrics({f"test_MA_{metric_name}_rounded":metric_value for metric_name, metric_value in test_metrics_MA.items()})


## Try POS without monitoring

In [208]:
unique_pos_ids = monitoring_reflex.CISLO_ODBERATELE.unique()

reflex_test = reflex_test[~reflex_test["CISLO_ODBERATELE"].isin(unique_pos_ids)]

reflex_test = reflex_test.filter(columns_filter_for_CATBoost)

remittence_test = reflex_test.REMITENDA

y_true = reflex_test.SALES

df_MA_model_copy = reflex_test.copy()
df_MA_model_copy["ROLLING_MEAN"] = df_MA_model_copy[["SALES_LAG_4","SALES_LAG_5","SALES_LAG_6","SALES_LAG_7"]].mean(axis=1)
MA_predictions = df_MA_model_copy.ROLLING_MEAN.round().values

reflex_test = reflex_test.drop(columns=["REMITENDA","SALES"])

In [209]:
logged_model = 'runs:/11824f1c145b4ba48937b4f54fd4a078/pipeline'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
y_pred = loaded_model.predict(pd.DataFrame(reflex_test))

y_pred = y_pred.round()

In [210]:
test_metrics_model = get_metrics(
        y_true=y_true, 
        y_pred=y_pred, 
        remittence=remittence_test)
    
test_metrics_model

{'MAE': 0.8362287598275425,
 'MSE': 1.8836545777326907,
 'r2': 0.8106295828896634,
 'hackathon_loss': 1.8756657367486684}

In [211]:
test_metrics_MA = get_metrics(
        y_true=y_true, 
        y_pred=MA_predictions, 
        remittence=remittence_test)
    
test_metrics_MA

{'MAE': 0.9079064164341871,
 'MSE': 2.2367803702764393,
 'r2': 0.7751286054721798,
 'hackathon_loss': 2.2034301293431398}